# Compustat에서 필요한 firm-lv 변수 뽑기

### 1. gvkey, year selection
- app_cit에 있는 기업만 고르기

### 2. 단순 변수 선택 & 제작
- ROA (ebit / at)
- 이익률 (ebit / revt)
- R&D intensity (xrd / revt)

### 3. market concentration
- 이건 전체 compustat 데이터로 비교 필요
- sic 코드 같은 애들 싹 다 모은 후 구하기

### 1. selection

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math

os.chdir('E:/apps')

In [2]:
raw = pd.read_csv('db_table/compustat_05-12.csv')

bio = pd.read_csv('appcit_bio.csv')
bio = bio.rename(columns = {'year':'fyear'})
bio.drop('app_cit', axis=1, inplace=True)

ee = pd.read_csv('appcit_EE&IT.csv')
ee = ee.rename(columns = {'year':'fyear'})
ee.drop('app_cit', axis=1, inplace=True)

len(bio), len(ee)

(1445, 2665)

In [3]:
bio = bio.merge(raw,how='left',on=['gvkey','fyear'])
ee = ee.merge(raw,how='left',on=['gvkey','fyear'])

len(bio), len(ee)

(1445, 2665)

In [7]:
bio.to_csv('compu_bio.csv', index=False)
ee.to_csv('compu_EE&IT.csv',index=False)

### 2. simple feature 제작
- ROA : ebit, at 없는 애들 못 구함
- 이익률 : ebit, revt 없는 애들 못 구함
- R&D intensity : xrd, revt 없는 애들 못 구함  
=> dropna on [ebit, at, revt, (xrd)]

__2.1 drop na__

In [4]:
list(bio)

['gvkey',
 'fyear',
 'conm',
 'consol',
 'datadate',
 'fic',
 'curcd',
 'act',
 'at',
 'ebit',
 'emp',
 'fatb',
 'fate',
 'fatl',
 'fatp',
 'fopo',
 'fsrco',
 'fsrcopo',
 'fsrcopt',
 'fsrct',
 'fuseo',
 'fuset',
 'ppegt',
 'ppent',
 'revt',
 'teq',
 'xlr',
 'xrd',
 'xstfws',
 'ni',
 'sic']

In [5]:
colnm = list(bio)
print colnm.index('fatb'), colnm.index('fuset')

11 21


In [6]:
bio.drop(colnm[11:22], axis=1, inplace=True)
ee.drop(colnm[11:22], axis=1, inplace=True)

In [7]:
bio1 = bio.dropna(axis=0, how='any',subset=['at','revt','ebit'])
bio2 = bio.dropna(axis=0, how='any',subset=['at','revt','ebit','xrd'])

print len(bio), len(bio1), len(bio2)

1445 1341 1300


In [8]:
ee1 = ee.dropna(axis=0, how='any',subset=['at','revt','ebit'])
ee2 = ee.dropna(axis=0, how='any',subset=['at','revt','ebit','xrd'])

print len(ee), len(ee1), len(ee2)

2665 2542 2411


In [9]:
bio2.to_csv('compu_bio_na.csv', index=False)
ee2.to_csv('compu_EE&IT_na.csv',index=False)

__2.2 변수 제작__

In [2]:
bio = pd.read_csv('compu_bio_na.csv')
ee = pd.read_csv('compu_EE&IT_na.csv')

bio['roa'] = bio['ebit'] / bio['at']
bio['margin'] = bio['ebit'] / bio['revt']
bio['rdi'] = bio['xrd'] / bio['revt']

ee['roa'] = ee['ebit'] / ee['at']
ee['margin'] = ee['ebit'] / ee['revt']
ee['rdi'] = ee['xrd'] / ee['revt']

In [5]:
bio.to_csv('compu_bio_temp.csv', index=False)
ee.to_csv('compu_EE&IT_temp.csv',index=False)